In [ ]:
!pip install category_encoders

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
# separete train data to X, y
X_train = df_train[["1stFlrSF", "TotalBsmtSF", "GarageArea", "GarageCars", "GrLivArea", "OverallQual"]]
y_train = df_train["SalePrice"]

In [ ]:
import lightgbm as lgb

In [ ]:
# LightGBMのモデル構築
params = {
    "objective": "regression",
    "metric": "rmse",  # RMSEを評価指標に使用
    "boosting_type": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": 0
}

In [ ]:
X_train_pra, X_train_val, y_train_pra, y_train_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
train_data = lgb.Dataset(X_train_pra, label=y_train_pra)
eval_data = lgb.Dataset(X_train_val, label=y_train_val, reference=train_data)

In [ ]:
model = lgb.train(params, train_data, valid_sets=[eval_data], num_boost_round=1000)

In [ ]:
y_train_pred = model.predict(X_train_val)
r2_score(y_train_val, y_train_pred)


0.7340583312202127

In [ ]:
X_test = df_test[["1stFlrSF", "TotalBsmtSF", "GarageArea", "GarageCars", "GrLivArea", "OverallQual"]]
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   1stFlrSF     1459 non-null   int64  
 1   TotalBsmtSF  1458 non-null   float64
 2   GarageArea   1458 non-null   float64
 3   GarageCars   1458 non-null   float64
 4   GrLivArea    1459 non-null   int64  
 5   OverallQual  1459 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 68.5 KB


In [ ]:
X_test.fillna(X_test.median(), inplace=True)

<ipython-input-14-5a869229f5f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_test.median(), inplace=True)


In [ ]:
y_test_pred = model.predict(X_test)
y_test_pred


array([132449.43932028, 150318.30303684, 139273.70652243, ...,
       158770.41283331, 101273.03736629, 245313.27726703])

In [ ]:
submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': y_test_pred})
submission.to_csv('submission_Advanced_Housing_Price_03.csv', index=False)